In [1]:
import tripyview.sub_notebookheader as nb_setup
# which_matplotlib = 'inline' (default), 
#                    'notebook'(jupyter notebook), 
#                    'widget'(jupyterlab)
nb_setup.init_notebook(which_matplotlib="inline")
# centralized autoimport of: 
# import os
# import warnings
# import time as clock
# import numpy as np
# import xarray as xr
# import shapefile as shp
# import tripyview as tpv
# client, use_existing_client = None, "tcp://0.0.0.0:0000"
# use_existing_client='tcp://127.0.0.1:42761'

/albedo/home/pscholz/tripyview/tripyview/sub_data.py:8: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater as sw


/albedo/home/pscholz/tripyview


### INFO:
To compute especially on large unstructured grids #vertices>1M, you need to run this notebook in parallel (do_parallel=True) on several workers (parallel_nprc...is the number of dask worker that can be allocated, parallel_tmem...is the maximum available RAM that will be distributed between the dask workers). Therefor allocate a full !!! COMPUTE NODE !!! (admins might not be happy if you do this in parallel on a login node) of a HPC of your choice with as much memory (RAM) as you can get to run this notebook!

In [2]:
# parameters
#___Dask Client Parameters____________________________________________________________
do_papermill      = False
do_parallel       = True
parallel_nprc     = 72   # number of total processes
parallel_nthread  = 1    # number of threads per worker --> number worker = parallel_nprc/parallel_nthread
# parallel_nprc     = 64   # number of total processes
# parallel_nthread  = 2    # number of threads per worker --> number worker = parallel_nprc/parallel_nthread

parallel_tmem     = 256  # max. available RAM

#___Mesh Path & Save Path_____________________________________________________________
# mesh_path ='/work/ollie/projects/clidyn/FESOM2/meshes/core2/'
mesh_path         = '/albedo/work/user/pscholz/mesh_fesom2.0/core2_pool/'
# mesh_path         = '/albedo/work/user/pscholz/mesh_fesom2.0/mesh_coast_500km_200m_10km/'

save_path         = None #'~/tripyview/Results/cstepanek/'
save_fname        = None # filename from papermill come in through save_fname
tripyrun_name     = None # papermill workflow name of notebook 
tripyrun_analysis = None # papermill diagnostic driver
tripyrun_spath_nb = None # papermill path to processed notebooks
tripyrun_spath_fig= None # papermill path to processed figures

#___Data Path & Input Names___________________________________________________________
input_paths   = list()
input_paths.append('/isibhv/projects/paleo_work/stepanek/velocity_plots_Wendy/data_raw/PI/')

input_names   = list()
input_names.append('PI')

# n_cycl: which spinupcycle should be plottet if do_allcycl all spinupcycles from [1...n_cycle] are plottet, if None path is directly used
n_cycl      = None
do_allcycl  = False
vname       = 'vec+u+v'
year        = [6901, 6999]
mon         = None
day         = None
record      = None 
box         = None
depth       = None
# do_data_r2g = True

# #___Define Reference Data, Year, Mon ...______________________________________________
# # do anomaly plots in case ref_path is not None
# ref_path  = None # '/albedo/work/projects/p_fesom/pscholz/project_TRR181/trr181_tke_ctrl_ck0.1/5/'
# ref_name  = None # 'TKE'
# ref_year  = None #[1979, 2019]
# ref_mon   = None
# ref_day   = None
# ref_record= None

# #___Define Climatology________________________________________________________________
# which_clim= 'phc3'
# clim_path = '/albedo/work/projects/p_fesom/FROM-OLLIE/FESOM2/hydrography/phc3.0/phc3.0_annual.nc'
# # clim_path = '/pool/data/AWICM/FESOM2/INITIAL/phc3.0/phc3.0_annual.nc'

# #___Define Colormap Parameters________________________________________________________
# # papermill doesnt like multi variable alignment in a single line
# cstr      = 'blue2red'
# cnum      = 15
# cref      = None
# crange    = None
# cmin      = None
# cmax      = None
# cfac      = None
# climit    = None
# chist     = True
# ctresh    = 0.995

# ref_cstr  = 'wbgyr'
# ref_cnum  = 15
# ref_cref  = None
# ref_crange= None
# ref_cmin  = None
# ref_cmax  = None
# ref_cfac  = None
# ref_climit= None
# ref_chist = True
# ref_ctresh= 0.995

# #___Define Plot Parameters____________________________________________________________
# ncol              = 2      # number of pannel columns in figure
# nrow              = None
# proj              = 'pc' 
# box               = [-180, 180, -90, 90]
# do_plt            = 'tpc' #'tcf'  # plot pcolor (tpc) or contourf (tcf)
# plt_contb         = False   # background contour line (thin)
# plt_contf         = False  # contour line of main colorbar steps 
# plt_contr         = False  # contour line of reference value 
# plt_contl         = False  # label contourline of main colorbar steps 
# do_rescale        = None   # rescale data: None, 'log10', 'slog10', np.array(...)
# do_lsm            ='fesom' # 'fesom', 'bluemarble', 'etopo', 'stock'
# do_mesh           = False, 
# mesh_opt          = dict({'color':'k', 'linewidth':0.10})
# do_enum           = False  # do enumeration of panels
# do_reffig         = True   # plot reference fig when doing anomalies 
# do_clim           = False   # plot climatolgy values when doing absoluts
# ax_title          = None
# cb_label          = None
# save_dpi          = 150
# save_fmt          = ['png']

In [3]:
# start parallel dask client if do_parallel=True
client = tpv.shortcut_setup_daskclient(client, 
                                       use_existing_client, 
                                       do_parallel, 
                                       parallel_nprc, 
                                       parallel_tmem,
                                       threads_per_worker=parallel_nthread)

No existing Dask cluster found at: tcp://0.0.0.0:0000
Started a new Dask client: <Client: 'tcp://127.0.0.1:35295' processes=72 threads=72, memory=214.58 GiB>


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: //proxy/8787/status,
Dashboard: //proxy/8787/status,Workers: 72
Total threads: 72,Total memory: 214.58 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35295,Workers: 72
Dashboard: //proxy/8787/status,Total threads: 72
Started: Just now,Total memory: 214.58 GiB
Comm: tcp://127.0.0.1:43545,Total threads: 1
Dashboard: //proxy/37263/status,Memory: 2.98 GiB
Nanny: tcp://127.0.0.1:38819,


In [4]:
#___LOAD FESOM2 MESH___________________________________________________________________________________
mesh=tpv.load_mesh_fesom2(mesh_path, do_rot='None', focus=0, do_info=True, do_pickle=True)

#______________________________________________________________________________________________________
# create input_path spinupcycle structure
# input_paths,· input_names, ref_path, ref_name = tpv.shortcut_setup_pathwithspinupcycles(input_paths, input_names, ref_path, ref_name, n_cycl, do_allcycl)

# #______________________________________________________________________________________________________
# cinfo=tpv.set_cinfo(cstr, cnum, crange, cmin, cmax, cref, cfac, climit, chist, ctresh)
# ref_cinfo=None
# if (ref_path != None): 
#     if ref_year   is None: ref_year   = year
#     if ref_mon    is None: ref_mon    = mon
#     if ref_record is None: ref_record = record
#     cinfo['cref']=0.0 
#     ref_cinfo=tpv.set_cinfo(ref_cstr, ref_cnum, ref_crange, ref_cmin, ref_cmax, ref_cref, ref_cfac, ref_climit, ref_chist, ref_ctresh)
    
# #______________________________________________________________________________________________________    
# # concatenate ref_path and input_path together if is not None,  concatenate list = list1+list2
# input_paths, input_names = tpv.shortcut_setup_concatinputrefpath(input_paths, input_names, ref_path, ref_name)

# #______________________________________________________________________________________________________
# # define index regions --> reading shape files
# box = tpv.shortcut_setup_boxregion(box_region)

#______________________________________________________________________________________________________
# use number of worker dependent chunk size for nodes and elements
chunks = dict({'time' : 'auto'    , 
               'elem' : mesh.n2de, #'auto', 
               'nod2' : mesh.n2dn, #'auto', 
               'nz1'  : 5, #mesh.nlev, 
               'nz'   : 5, #mesh.nlev, 
               'edg_n': 'auto'}) 

if client is not None:
    # set size limit of time chunks so that size of the total chunks covers 10% of worker memory
    mem_thresh     = 0.2
    worker_info    = client.run_on_scheduler(lambda dask_scheduler: {worker: ws.memory_limit for worker, ws in dask_scheduler.workers.items()})
    mem_limit_byte = next(iter(worker_info.items()))[1]
    print(' --> worker memory      : {:04.2f}Mb'.format(mem_limit_byte/1024**2))
    print(' --> limit chunksize to : {:04.2f}Mb'.format(mem_limit_byte/1024**2*mem_thresh))
    chunk_size = mem_limit_byte * mem_thresh/4 # dive by 4 because we use 4byte single precision
    if   chunks['nod2'] != 'auto': chunk_size = chunk_size/chunks['nod2']    
    elif chunks['nod2'] == -1    : chunk_size = chunk_size/mesh.n2dn
    elif chunks['elem'] != 'auto': chunk_size = chunk_size/chunks['elem']
    elif chunks['elem'] == -1    : chunk_size = chunk_size/mesh.n2de
    if   chunks['nz'  ] != 'auto': chunk_size = chunk_size/chunks['nz']
    elif chunks['nz'  ] == -1    : chunk_size = chunk_size/mesh.nlev
    elif chunks['nz1' ] != 'auto': chunk_size = chunk_size/chunks['nz1']
    elif chunks['nz1' ] == -1    : chunk_size = chunk_size/mesh.nlev
    # elif chunks['time'] != 'auto': chunk_size = chunk_size/chunks['time']    
    chunks['time'] = np.int32(np.ceil(chunk_size))
    print(' --> set time chunk size to {:2.2f}% of worker mem.: {:d}'.format(100*mem_thresh, chunks['time']))

 > warning: pickle5 module could not be found, no do_pickle 
 is possible! Therefor switch to joblib saving/loading
 > found *.jlib file: /albedo/work/user/pscholz/mesh_fesom2.0/core2_pool
 > load  *.jlib file: tripyview_fesom2_core2_pool_focus0.jlib
___FESOM2 MESH INFO________________________
 > path            = /albedo/work/user/pscholz/mesh_fesom2.0/core2_pool
 > id              = core2_pool
 > do rot          = None
 > [al,be,ga]      = 50, 15, -90
 > do augmpbnd     = True
 > do cavity       = False
 > do lsmask       = True
 > do earea,eresol = True, False
 > do narea,nresol = True, False
___________________________________________
 > #node           = 126858
 > #elem           = 244659
 > #lvls           = 48
___________________________________________
 --> worker memory      : 3051.76Mb
 --> limit chunksize to : 610.35Mb
 --> set time chunk size to 20.00% of worker mem.: 253


### INFO:
do rotation of elemental velocites from rotated coordinates into geo-cordinates and do interpolation from elements to vertices to obtain unod and vnod. And save unod and vnod into new netcdf files. Do here the rotation and interpolation over chunks of year since its faster like that. Chunks can be choose based on the availablility of memory.
It is strongly recomented to do this in parallel via a dask client!!!

In [9]:
t0 = clock.time()
vname = 'vec+u+v'
vnameu_new, vnamev_new = 'unod', 'vnod'
output_path = '/albedo/scratch/user/pscholz/test_cstepanek_ie2n/'


# clean up garabage on workers before the party starts!
if client is not None: client.run(gc.collect)
    
data_list     = list()
year_list     = list(range(year[0], year[1]+1))
chnk_size     = 10  # compute in chunks of chnk_size years 
                    #-> you can use larger chunks here if you have enough memory
chnk_yrs_list = year_list[::chnk_size]

#___LOAD FESOM2 DATA___________________________________________________________
for ii, (datapath, descript) in enumerate(zip(input_paths, input_names)):
    print(ii, datapath, descript)
    print(' > year:', end='')
    for nchnk, year_s in enumerate(chnk_yrs_list): 
        ts = clock.time()
        year_chnk = [ year_s, np.min([year_s+chnk_size-1,year[1]]) ]  
        print(*np.arange(year_chnk[0],year_chnk[1]+1), sep='|',end='')
        
        #__________________________________________________________________________
        # create shortcut for input parameterm load several years at ones -->
        # much faster than loading every every separated
        input_dict = dict({'year':year_chnk, 'depth':depth, 
                           'descript':descript, 'do_info':False, 'do_ie2n':True, 'do_rot':True, 
                           'do_tarithm':None, 'do_zarithm':None, 
                           'chunks':chunks, 'do_parallel':do_parallel, 'do_load':True, 'do_persist':False, 
                           'client':client})

        data = tpv.load_data_fesom2(mesh, datapath, vname=vname, **input_dict).rename({'n_u':vnameu_new, 'n_v':vnamev_new})
        data_u = data[['unod']]
        data_v = data[['vnod']]
        
        #__________________________________________________________________________
        # save every year to netcdf
        for yeari in range(year_chnk[0], year_chnk[1]+1):
            sfnameu, sfnamev = '{:s}.fesom.{:d}.nc'.format(vnameu_new, yeari), '{:s}.fesom.{:d}.nc'.format(vnamev_new, yeari)
            data_u.sel(time=str(yeari)).to_netcdf(path=os.path.join(output_path,sfnameu))
            data_v.sel(time=str(yeari)).to_netcdf(path=os.path.join(output_path,sfnamev))
        
        if client is not None: client.run(gc.collect)
        gc.collect
        
        print(' --> total elasped time to process data: {:3.2f} min.'.format( (clock.time()-t0)/60  )) 
        

0 /isibhv/projects/paleo_work/stepanek/velocity_plots_Wendy/data_raw/PI/ PI
 > year:6901|6902|6903|6904|6905|6906|6907|6908|6909|6910

/albedo/home/pscholz/.conda/envs/py39_new/lib/python3.9/site-packages/xarray/coding/times.py:992: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/albedo/home/pscholz/.conda/envs/py39_new/lib/python3.9/site-packages/xarray/coding/times.py:992: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/albedo/home/pscholz/.conda/envs/py39_new/lib/python3.9/site-packages/xarray/core/indexing.py:526: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(self.get_duck_array(), dtype=dtype)
/albedo/hom

 > do vector rotation
 > do interpolation e2n


/albedo/home/pscholz/.conda/envs/py39_new/lib/python3.9/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 448.00 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


43.08209943771362
43.27889919281006
 --> total elasped time to process data: 1.64 min.


NameError: name 'STOP' is not defined

In [ ]:
if do_papermill and do_parallel and client is not None: client.shutdown()